Ollama can be used with GGML or GGUF. However GGUF is the successor of GGML due to some constrains source: [medium](https://medium.com/@phillipgimmi/what-is-gguf-and-ggml-e364834d241c)

In [54]:
import ollama
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM
import os
from dotenv import load_dotenv

In [32]:
load_dotenv()

True

In [ ]:
ollama.list()

In [45]:
MODEL_ID = "meta-llama/Llama-3.2-1B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
model: LlamaForCausalLM = AutoModelForCausalLM.from_pretrained(MODEL_ID, device_map="auto", trust_remote_code=True)

# ... Here could be some tuning code

In [48]:
model.save_pretrained("model")
tokenizer.save_pretrained("model")

('model\\tokenizer_config.json',
 'model\\special_tokens_map.json',
 'model\\tokenizer.json')

In [12]:
# Create the directory move into it and clone llama.cpp there
!mkdir dependencies && cd dependencies && git clone https://github.com/ggerganov/llama.cpp

Cloning into 'llama.cpp'...


In [17]:
!pip install -r dependencies/llama.cpp/requirements/requirements-convert_hf_to_gguf.txt

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.5.1+cu124 requires torch==2.5.1+cu124, but you have torch 2.2.2+cpu which is incompatible.
torchvision 0.20.1+cu124 requires torch==2.5.1+cu124, but you have torch 2.2.2+cpu which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------------ ------------------------- 20.5/61.0 kB 320.0 kB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 651.3 kB/s eta 0:00:00
     ---------------------------------------- 0.0/200.7 MB ? eta -:--:--
     ---------------------------------------- 0.1/200.7 MB ? eta -:--:--
     ---------------------------------------- 0.2/200.7 MB 3.1 MB/s eta 0:01:04
     ---------------------------------------- 0.4/200.7 MB 3.4 MB/s eta 0:00:59
     ---------------------------------------- 0.9/200.7 MB 5.1 MB/s eta 0:00:39
     ---------------------------------------- 1.5/200.7 MB 8.0 MB/s eta 0:00:25
      -------------------------------------- 2.8/200.7 MB 10.5 MB/s eta 0:00:19
      -------------------------------------- 3.9/200.7 MB 12.3 MB/s eta 0:00:16
      -------------------------------------- 5.1/200.7 MB 14

In [ ]:
!mkdir converted && python dependencies/llama.cpp/convert_hf_to_gguf.py model --outfile converted/gguf_model.gguf --outtype q8_0

INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {32}
INFO:hf-to-gguf:gguf: loading model part 'model.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.float32 --> Q8_0, shape = {2048, 128256}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.float32 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.float32 --> Q8_0, shape = {8192, 2048}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.float32 --> Q8_0, shape = {2048, 8192}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.float32 --> Q8_0, shape = {2048, 8192}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.float32 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.float32 --> Q8_0, shape = {2048, 512}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.float32 --> Q8_0, shape = {2048, 2048}
INFO:hf-to

In [55]:
model_location = os.path.abspath("converted/gguf_model.gguf")
with open("converted/Modelfile", "w+") as file:
    file.write(f"from {model_location}")

In [56]:
ollama.create("our_model", "./converted/Modelfile")

{'status': 'success'}

In [61]:
answer = ollama.generate("our_model", "What is 3 x 3 - 15?")
print(answer["response"])

 First, multiply the numbers 3 and 3. The product of 3 x 3 is 9. Then subtract 15 from 9. The result of this subtraction operation is 0. Therefore, 3 x 3 - 15 equals 0.


Model Formats explained

[Detailed Formats](https://vickiboykis.com/2024/02/28/gguf-the-long-way-around/#finally-gguf)

Alternative Conversion with Triton

[Native Conversion Unsloph](https://colab.research.google.com/drive/1T5-zKWM_5OD21QHwXHiV9ixTRR7k3iB9?usp=sharing#scrollTo=IqM-T1RTzY6C)

[Unsloph](https://github.com/unslothai/unsloth)

[Requirement](https://github.com/woct0rdho/triton-windows)